In [1]:
import numpy as np
from dolfinx import mesh, fem, io
from mpi4py import MPI
from petsc4py import PETSc
import ufl

# Parameters
length = 10e-6
height = 5e-6
nx = 100
ny = 50

# 2D mesh
domain = mesh.create_rectangle(MPI.COMM_WORLD,
                               [np.array([0, 0]), np.array([length, height])],
                               [nx, ny],
                               cell_type=mesh.CellType.triangle)

# Function space
V = fem.functionspace(domain, ("CG", 1))

# Relative permittivity (scalar)
eps_r = fem.Constant(domain, PETSc.ScalarType(28.0))  # Approx LiNbO3

# Boundary markers
def left_boundary(x): # so this is linteraly markers. if it is close to zero, it is the left boundary
    """Marker for the left boundary."""
    return np.isclose(x[0], 0.0)

def right_boundary(x):
    return np.isclose(x[0], length)

facets_left = mesh.locate_entities_boundary(domain, dim=1, marker=left_boundary)
facets_right = mesh.locate_entities_boundary(domain, dim=1, marker=right_boundary)

# Dirichlet Boundry Conditions
# Left boundary at 100V, right boundary at 0V
dofs_left = fem.locate_dofs_topological(V, entity_dim=1, entities=facets_left)
dofs_right = fem.locate_dofs_topological(V, entity_dim=1, entities=facets_right)
# comuting degrees of freedom for the left and right boundaries

bc_left = fem.dirichletbc(PETSc.ScalarType(100.0), dofs_left, V)
bc_right = fem.dirichletbc(PETSc.ScalarType(0.0), dofs_right, V)
# Note: The Dirichlet boundary conditions are set to 100V on the left and 0V on the right.

# Variational problem.
u = ufl.TrialFunction(V)
v = ufl.TestFunction(V)

a = eps_r * ufl.dot(ufl.grad(u), ufl.grad(v)) * ufl.dx(domain)
# Linear form (homogeneous RHS)
zero = fem.Constant(domain, PETSc.ScalarType(0.0))
L = zero * v * ufl.dx(domain)

# LinearProblem setup
from dolfinx.fem.petsc import LinearProblem

problem = LinearProblem(
    a,
    L,
    bcs=[bc_left, bc_right],
    petsc_options={"ksp_type": "cg"}
)

phi_sol = problem.solve()

In [2]:
# Compute E = -grad(phi)
E = fem.Expression(-ufl.grad(phi_sol), V.element.interpolation_points())

In [3]:
# # Export to XDMF
# with io.XDMFFile(MPI.COMM_WORLD, "potential.xdmf", "w") as xdmf:
#     xdmf.write_mesh(domain)
#     phi_sol.name = "Potential"
#     xdmf.write_function(phi_sol)

# Save E-field components
# Note: Since E is a vector expression, project each component
W = fem.functionspace(domain, ("CG", 1))
grad_phi = ufl.grad(phi_sol)

# Project Ex
Ex = fem.Function(W)
ex_form = -grad_phi[0]
a_proj = ufl.inner(ufl.TestFunction(W), ufl.TrialFunction(W)) * ufl.dx
L_proj_ex = ufl.inner(ufl.TestFunction(W), ex_form) * ufl.dx
problem_ex = LinearProblem(a_proj, L_proj_ex, petsc_options={"ksp_type": "cg"})
Ex.x.array[:] = problem_ex.solve().x.array

# Project Ey
Ey = fem.Function(W)
ey_form = -grad_phi[1]
L_proj_ey = ufl.inner(ufl.TestFunction(W), ey_form) * ufl.dx
problem_ey = LinearProblem(a_proj, L_proj_ey, petsc_options={"ksp_type": "cg"})
Ey.x.array[:] = problem_ey.solve().x.array

# with io.XDMFFile(MPI.COMM_WORLD, "efield.xdmf", "w") as xdmf:
#     xdmf.write_mesh(domain)
#     Ex.name = "Ex"
#     Ey.name = "Ey"
#     xdmf.write_function(Ex)
#     xdmf.write_function(Ey)

In [4]:
r13 = 10
r22 = 6.8
r33 = 32.2
r51 = 32.0

r_voigt = np.array([
    [0,    0,    0,     0,   r13, 0],
    [0,    0,    0,    r13,   0,  0],
    [r33, r33, r33,     0,   0,   0]
])

r_voigt *= 1e-12

# Refractive index (assuming uniaxial crystal)
n0 = 2.2

delta_n = fem.Function(V)

In [5]:
Ex

Coefficient(FunctionSpace(Mesh(blocked element (Basix element (P, triangle, 1, gll_warped, unset, False, float64, []), (2,)), 0), Basix element (P, triangle, 1, gll_warped, unset, False, float64, [])), 1)

In [1]:
import numpy as np
import dolfinx
from dolfinx import mesh, fem, io, default_scalar_type
from dolfinx.fem.petsc import LinearProblem
import ufl
from mpi4py import MPI
import basix

def simulate_electric_field():
    """
    Simulate uniform electric field in a 3mm x 3mm x 3mm cube
    Voltage differences: 2V across x and y axes, 5V across z axis
    """
    
    # Create 3D mesh - 3mm cube
    domain = mesh.create_box(MPI.COMM_WORLD, 
                           [[0.0, 0.0, 0.0], [3e-3, 3e-3, 3e-3]], 
                           [20, 20, 20], 
                           mesh.CellType.tetrahedron)
    
    # Define function space (first order Lagrange elements)
    V = fem.functionspace(domain, ("Lagrange", 1))
    
    # Define boundary conditions
    # For uniform field, we need linear potential distribution
    # V(x,y,z) = ax + by + cz + d
    # Given voltage differences:
    # - 2V across x (0 to 3mm): a = 2V / 3mm = 666.67 V/m
    # - 2V across y (0 to 3mm): b = 2V / 3mm = 666.67 V/m  
    # - 5V across z (0 to 3mm): c = 5V / 3mm = 1666.67 V/m
    
    def potential_function(x):
        """Define the analytical potential for uniform field"""
        return 666.67 * x[0] + 666.67 * x[1] + 1666.67 * x[2]
    
    # Apply boundary conditions on all faces
    # Bottom face (z=0)
    def bottom_boundary(x):
        return np.isclose(x[2], 0.0)
    
    # Top face (z=3mm)  
    def top_boundary(x):
        return np.isclose(x[2], 3e-3)
    
    # Left face (x=0)
    def left_boundary(x):
        return np.isclose(x[0], 0.0)
    
    # Right face (x=3mm)
    def right_boundary(x):
        return np.isclose(x[0], 3e-3)
    
    # Front face (y=0)
    def front_boundary(x):
        return np.isclose(x[1], 0.0)
    
    # Back face (y=3mm)
    def back_boundary(x):
        return np.isclose(x[1], 3e-3)
    
    # Create boundary condition objects
    bottom_dofs = fem.locate_dofs_geometrical(V, bottom_boundary)
    top_dofs = fem.locate_dofs_geometrical(V, top_boundary)
    left_dofs = fem.locate_dofs_geometrical(V, left_boundary)
    right_dofs = fem.locate_dofs_geometrical(V, right_boundary)
    front_dofs = fem.locate_dofs_geometrical(V, front_boundary)
    back_dofs = fem.locate_dofs_geometrical(V, back_boundary)
    
    # Define potential values at boundaries
    bottom_bc = fem.dirichletbc(default_scalar_type(0.0), bottom_dofs, V)  # V=0 at z=0
    top_bc = fem.dirichletbc(default_scalar_type(5.0), top_dofs, V)        # V=5V at z=3mm
    left_bc = fem.dirichletbc(default_scalar_type(0.0), left_dofs, V)      # V=0 at x=0
    right_bc = fem.dirichletbc(default_scalar_type(2.0), right_dofs, V)    # V=2V at x=3mm
    front_bc = fem.dirichletbc(default_scalar_type(0.0), front_dofs, V)    # V=0 at y=0
    back_bc = fem.dirichletbc(default_scalar_type(2.0), back_dofs, V)      # V=2V at y=3mm
    
    # Note: This overconstrained system will find the best fit solution
    bcs = [bottom_bc, top_bc, left_bc, right_bc, front_bc, back_bc]
    
    # Define variational problem for Laplace equation: ∇²φ = 0
    u = ufl.TrialFunction(V)
    v = ufl.TestFunction(V)
    
    # Weak form: ∫∇u·∇v dx = 0
    a = ufl.dot(ufl.grad(u), ufl.grad(v)) * ufl.dx
    L = fem.Constant(domain, default_scalar_type(0.0)) * v * ufl.dx
    
    # Solve the linear system
    problem = LinearProblem(a, L, bcs=bcs, 
                          petsc_options={"ksp_type": "preonly", "pc_type": "lu"})
    phi = problem.solve()
    
    # Calculate electric field E = -∇φ
    W = fem.functionspace(domain, ("Lagrange", 1, (domain.geometry.dim,)))
    E_expr = fem.Expression(-ufl.grad(phi), W.element.interpolation_points())
    E = fem.Function(W)
    E.interpolate(E_expr)
    
    # Output results
    print("Electric Field Simulation Results:")
    print("=" * 40)
    print(f"Domain: 3mm × 3mm × 3mm cube")
    print(f"Voltage difference across x: 2V")
    print(f"Voltage difference across y: 2V") 
    print(f"Voltage difference across z: 5V")
    
    # Calculate field magnitudes at center point
    center_point = np.array([[1.5e-3, 1.5e-3, 1.5e-3]])
    try:
        E_center = E.eval(center_point, [0])
        print(f"\nElectric field at center (V/m):")
        print(f"Ex = {E_center[0]:.1f}")
        print(f"Ey = {E_center[1]:.1f}")
        print(f"Ez = {E_center[2]:.1f}")
        print(f"|E| = {np.linalg.norm(E_center):.1f}")
    except:
        print("\nNote: Field evaluation at specific point requires interpolation")
    
    # Expected uniform field values
    print(f"\nExpected uniform field (V/m):")
    print(f"Ex = 666.7 (2V/3mm)")
    print(f"Ey = 666.7 (2V/3mm)")
    print(f"Ez = 1666.7 (5V/3mm)")
    print(f"|E| = {np.sqrt(666.7**2 + 666.7**2 + 1666.7**2):.1f}")
    
    # Save results to files
    with io.VTXWriter(domain.comm, "potential.bp", [phi]) as vtx:
        vtx.write(0.0)
    
    with io.VTXWriter(domain.comm, "electric_field.bp", [E]) as vtx:
        vtx.write(0.0)
    
    print(f"\nResults saved to:")
    print(f"- potential.bp (electric potential)")
    print(f"- electric_field.bp (electric field vector)")
    
    return phi, E, domain

def visualize_results(phi, E, domain):
    """
    Optional visualization using PyVista (requires pyvista installation)
    """
    try:
        import pyvista as pv
        
        # Create pyvista grid from dolfinx mesh
        topology, cell_types, geometry = dolfinx.plot.vtk_mesh(domain, domain.topology.dim)
        grid = pv.UnstructuredGrid(topology, cell_types, geometry)
        
        # Add potential data
        grid.point_data["Potential"] = phi.x.array.real
        
        # Add electric field data  
        grid.point_data["Electric_Field"] = E.x.array.real.reshape((-1, 3))
        
        # Create plotter
        plotter = pv.Plotter(shape=(1, 2))
        
        # Plot potential
        plotter.subplot(0, 0)
        plotter.add_mesh(grid, scalars="Potential", show_edges=True)
        plotter.add_text("Electric Potential", font_size=12)
        
        # Plot electric field magnitude
        plotter.subplot(0, 1)
        grid.point_data["E_magnitude"] = np.linalg.norm(
            grid.point_data["Electric_Field"], axis=1)
        plotter.add_mesh(grid, scalars="E_magnitude", show_edges=True)
        plotter.add_text("Electric Field Magnitude", font_size=12)
        
        plotter.show()
        
    except ImportError:
        print("\nNote: Install pyvista for 3D visualization:")
        print("pip install pyvista")

if __name__ == "__main__":
    # Run simulation
    phi, E, domain = simulate_electric_field()
    
    # Uncomment to visualize (requires pyvista)
    # visualize_results(phi, E, domain)

ld: warning: duplicate -rpath '/opt/anaconda3/envs/fenicsx-env/lib' ignored
ld: warning: duplicate -rpath '/opt/anaconda3/envs/fenicsx-env/lib' ignored
ld: warning: duplicate -rpath '/opt/anaconda3/envs/fenicsx-env/lib' ignored
ld: warning: duplicate -rpath '/opt/anaconda3/envs/fenicsx-env/lib' ignored
ld: warning: duplicate -rpath '/opt/anaconda3/envs/fenicsx-env/lib' ignored
ld: warning: duplicate -rpath '/opt/anaconda3/envs/fenicsx-env/lib' ignored


Electric Field Simulation Results:
Domain: 3mm × 3mm × 3mm cube
Voltage difference across x: 2V
Voltage difference across y: 2V
Voltage difference across z: 5V

Electric field at center (V/m):
Ex = -88817.2
Ey = 208817.2
Ez = 0.0
|E| = 226921.0

Expected uniform field (V/m):
Ex = 666.7 (2V/3mm)
Ey = 666.7 (2V/3mm)
Ez = 1666.7 (5V/3mm)
|E| = 1914.9

Results saved to:
- potential.bp (electric potential)
- electric_field.bp (electric field vector)
